In [1]:
import pandas as pd

import numpy as np

from scipy import sparse

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

Neste notebook, vamos explorar um pouco a Expansão de Consultas na Recuperação de Informação. A Expansão de consultas é uma ferramenta que nos auxilia a melhorar a consulta passada pelo usuário, através da análise da relação entre as palavras da consulta e as palavras que vem depois delas nos documentos. Assim, podemos inferir que a palavra que mais aparece depois de outra tem uma forte ligação com ela.

Primeiro, vamos montar a matriz de coocorrência. As linhas e colunas dessa matriz são as palavras do nosso dicionário, e a relação matriz[palavra1][palavra2] nos da a quantidade de vezes que a palavra 2 apareceu imediatamente depois da palavra1.

In [2]:
documentos = pd.read_csv("../../modelo_vetorial/estadao_noticias_eleicao.csv")
documentos = documentos.replace(np.nan, '', regex=True)

In [3]:
content = documentos.titulo + " " + documentos.subTitulo + " " + documentos.conteudo
content = content.fillna("")

In [4]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [5]:
# Removendo pontuação
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

In [8]:
# Removendo stopwords
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

In [9]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [10]:
matrix, vocab = co_occurrence_matrix(tokens)

In [11]:
coocorrenciaMatrix = matrix.tocsr()

In [12]:
vocabIds = {v: k for k, v in vocab.items()}

A função abaixo expande o termo passado como parâmetro, retornando os termos que aparecem com mais frequência depois dele.

In [13]:
'''
Função que expande o termo passado como parâmetro para os termos mais
próximos a ele.

@param termo - termo a ser expandido
@param numeroDeExpansoes quantidade de termos a serem expandidos

@returns Array expansão dos termos
'''
def expande_termo(termo, numeroDeExpansoes):
    linha = coocorrenciaMatrix[vocab[termo]].toarray()
    termosExpandidos = np.argpartition(linha[0], -numeroDeExpansoes)[-numeroDeExpansoes:]
    expansao = []
    for idTermo in termosExpandidos:
        expansao.append(vocabIds[idTermo])
    return expansao

Agora que temos a matriz de coocorrência dos termos e a função que realiza a expansão dos termos, vamos montar o nosso índice invertido com os documentos apresentados. A implementação será a mesma utilizada nos labs passados.

In [14]:
def geraIndice(documentos):
    indice_invertido = dict()
    for i, row in documentos.iterrows():
        
        titulo = (word.lower() for word in (nltk.word_tokenize(row['titulo'])))
        subtitulo = (word.lower() for word in (nltk.word_tokenize(row['subTitulo'])))
        conteudo = (word.lower() for word in (nltk.word_tokenize(row['conteudo'])))
        palavras_divididas = list(titulo) + list(subtitulo) + list(conteudo)
        for palavra in palavras_divididas:
            if palavra not in indice_invertido:
                indice_invertido[palavra] = set([row['idNoticia']])
            else:
                indice_invertido[palavra].add(row['idNoticia'])
    
    return indice_invertido

In [15]:
indice_invertido = geraIndice(documentos)

Para este lab, vamos intanciar o modelo binário. As consultas realizadas serão a disjunção dos termos.

In [16]:
def consultaOR(termos):
    indice = []
    for termo in termos:
        indice.append(indice_invertido[termo])
    return list(set.intersection(*indice))

In [17]:
def consultaExpandida(termos):
    termosExpansao = []
    for termo in termos:
        termosExpansao = termosExpansao + expande_termo(termo, 3)
    return termos + termosExpansao

Agora que temos uma função que expande os termos da consulta, vamos escolher três termos e ver como fica a consulta expandida pra eles. Os termos serão PT, Petrobrás e Lula.

In [18]:
print("PT: ")
print(expande_termo("pt", 3))
print("Petrobrás")
print(expande_termo("petrobrás", 3))
print("Lula") 
print(expande_termo("lula", 3))

PT: 
['pmdb', 'governo', 'psdb']
Petrobrás
['graça', 'é', 'paulo']
Lula
['disse', 'dilma', 'silva']


Como podemos ver, os termos expandidos são extremamente relacionados com o termo original. PMDB, PSDB são partidos políticos assim como o PT. O nosso filtro não funcionou muito bem com as stopwords, tendo trazido o "é" para Petrobrás. Já Lula também teve termos muito relacionados, como Dilma, a presidenta que o sucedeu, Silva, seu sobrenome. 

Vamos agora realizar uma consulta com os termos expandidos e sem para ver como fica.

In [20]:
consultaOriginal = ["presidente", "lula"]
documentosSemExpansao = consultaOR(consultaOriginal)
documentosComExpansao = consultaOR(consultaExpandida(consultaOriginal))

A consulta escolhida foi "Presidente Lula". Veremos agora quais os documentos retornados por cada consulta:

In [30]:
print("Sem expansão")
print(documentosSemExpansao[:5])
print("\n")
print("Com expansão")
print(documentosComExpansao[:5])

Sem expansão
[1, 2, 3, 8197, 8198]


Com expansão
[6148, 4619, 6156, 4112, 4625]


Como podemos ver, nenhum documento retornado na consulta original está na consulta expandida. Isso acontece por que a quantidade de documentos que é retornada na consulta expandida é bem maior que na consulta original. Isso deve-se ao fato que a consulta OR é a união dos conjuntos dos documentos que contém algum dos termos.

A expansão é interessante por trazer mais documentos que provavelmente estão relacionados aos termos originalmente requisitados. Se utilizássemos outro modelo, como o BM25, por exemplo, para rankear os documentos, provavelmente obteriamos resultados melhor utilizando a expansão.

De modo geral, a expansão ajuda a melhorar o recall por trazer termos que estão relacionados a consulta original,